#  Activity 3.0: Vectorization

This mini activity is designed to help you get more comfortable with vectorization.  There are 3 total questions in two parts.

**Due date**: Thursday October 5th, 2023, 9:00 pm EDT.

**Instructions for Submission**: Submit via Gradescope.

## Part 1: Reading Vectorized Code

In this problem you need to read, understand and explain what is going on in a few snippets of code that the compiler generated for the problem we discussed in class.  You will want to look up each instruction to understand what it is doing and what the cost is.

The code and assembly can be seen [https://godbolt.org/z/d3oKW4K3E](https://godbolt.org/z/d3oKW4K3E)

### Question 1
We first look at the loop body 

```
.LBB0_6:
        vpcmpeqd        xmm3, xmm1, xmmword ptr [rdi + 4*rax]
        vpmovzxdq       ymm3, xmm3              # ymm3 = xmm3[0],zero,xmm3[1],zero,xmm3[2],zero,xmm3[3],zero
        vpand   ymm3, ymm3, ymm2
        vpaddq  ymm0, ymm0, ymm3
        add     rax, 4
        cmp     rcx, rax
        jne     .LBB0_6
```

Please explain what this is doing, and how many cycles each iteration of the loop takes.  

Your explanation should include what are the inputs (what values are in each register at the beginning), what are the outputs (the values of the registers at the end), and how it is computing this.  Your answer should be complete in that all 7 instructions must be explained.


1. The vpcmpeqd command compares the two vectors registers xmm1, containing the value of target multiple times, and the register xmmword ptr \[pdi + 4*rax\], which contains a vector with 4 consecutive elements of the data array, for if they are equal or not. If the vectors have data elements equal, then the corresponding data element is set to 1s in xmm3 and 0s otherwise. This costs 1 cycles. 
2. The vpmozxdq moves the result from the xmm3 vector register to the ymm3 vector register which is double the size. It does this by taking each data element from the xmm3 vector register and padding it with zeroes until it is quad size and then storing that in the corresponding data element in the ymm3 vector register. This costs 3 cycles
3. The vpand performs an and on each data element on ymm3 and ymm2 and stores it in ymm3. ymm2 is the vector where each data element is 1. Essentially, if a data element of ymm3 has 1s, meaning it matched with the target, it would get anded with 1, leaving just 1. Otherwise, the output would just be 0. This costs 1 cycles
4. The vpaddq command adds ymm0 and ymm3 and puts the result into ymm0. ymm0 is the vector register that is accumulating the result and ymm3 is the result of the vpand command. This operation accumulates the result where the number in the first data element of ymm3 is equal to the count of all the elements in the data array that is equal to target where its index is equal to 0 mod 4. Similarly, the next data element is the count of all elements in the data array equal to target where its index is equal to 1 mod 4 etc. This costs 1 cycles
5. The counter variable stored in the register rax is incremented by 4. This is done since our vectorized instruction does operations on 4 elements of data at a time. This costs 1 cycle.
6. Compares the counter variable rax with the condition rcx which contains the value of n. Sets flags depending on the value of the comparison. This costs 1 cycle.
7. Looks at the flags and if our counter rax is not equal to rcx which is n, then we will jump back to the top of the loop and repeat. This takes 1? cycles.

The total number of cycles required is about 8 cycles.

### Question 2

After the loop

```
        vextracti128    xmm1, ymm0, 1
        vpaddq  xmm0, xmm0, xmm1
        vpshufd xmm1, xmm0, 238                 # xmm1 = xmm0[2,3,2,3]
        vpaddq  xmm0, xmm0, xmm1
        vmovq   rax, xmm0
```

Please explain what this is doing, and how many cycles it takes.  

Once again please include the inputs, outputs, and cost.  You must include an explanation for every instruction and what it is doing.

1. vextracti128 extracts the values from the vector register ymm0, which contains the accumulated counts, into a smaller vector register xmm1. This discards the padded 0s. This takes 4 cycles.
2. Adds xmm1 to xmm0 and stores the result in xmm0. xmm0 is a vector register with all 0s since it was xored with itself earlier. This results in xmm0 having the same value as xmm1. This takes 1 cycles. 
3. Shuffles the value of xmm0 and inserts it into xmm1 such that the first data element of xmm1 is the third data element of xmm0, the second data element of xmm1 is the fourth data element of xmm0, the third data element of xmm1 is the third data element of xmm0, and finally the fourth data element of xmm1 is the fourth data element of xmm0. This takes 6 cycles.
4. Adds xmm1 and xmm0 together. Since xmm0 has the original counts of the data elements and xmm1 has counts with only the third and fourth counts, this operation results in summing up all the counts as the counts from the first and second data elements from xmm0 gets added with the counts from the first and second data elements of xmm1 which were the third and fourth counts. This takes 1 cycles
5. This operation moves the result of the total counts into the return register. This takes 3 cycles.

The total cost is 15 cycles.

## Part 2: Writing Vectorized Code

In this part you will tackle a new problem, write some code for it, and then analyze it.  The problem can be found at [http://preview.speedcode.org/ide/index.html?count_pairs](http://preview.speedcode.org/ide/index.html?count_pairs)

The goal of the problem us to count unaligned pairs of bytes in an array.

The starting code is 
```c
uint64_t count_pairs(uint8_t *data, uint64_t size, uint8_t target) {
  uint64_t total = 0;
  for (uint64_t i = 0; i < size - 1; i++) {
    if (data[i] == target && data[i + 1] == target) {
      total += 1;
    }
  }
  return total;
}
```




### Question 3

Please achieve 1,000% speedup or more over the reference code and include your code in your submission.

You must explain your solution in English as well.  Submissions without a full explanation will not receive points. 

If you did it using intrinsics then explain your inner loop as you did for the previous problem.  
Including: 
 - how does it compute the answer?
 - how many cycles does it take?
 - how many iterations of the base loop from the starting code does it compute on each iteration?

If you did it without using intrinsics please explain what you did to transform the problem into a form that the compiler could vectorize.

Yes, this is a hint that it can be done either with, or without intrinsics


```cpp
#include "solution.hpp"
#include <cilk/cilk.h>
#include <cilk/opadd_reducer.h>

// ======================================================================
// SOLUTION ENTRY
// ======================================================================
void zero_int(void *view) { *(uint64_t *)view = 0; }
void add_int(void *left, void *right) { *(uint64_t *)left += *(uint64_t *)right; }
uint64_t count_pairs(uint8_t *data, uint64_t size, uint8_t target) {
  uint64_t cilk_reducer(zero_int, add_int) total = 0;
  uint64_t clean_end = ((size - 1)/32)*32;

  auto target_vec = _mm256_set1_epi8(target);
  cilk_for (uint64_t i=0; i < clean_end; i+=32) {
      auto cmp_vec1 = _mm256_cmpeq_epi8(target_vec, _mm256_loadu_si256((__m256i *)(data + i)));
      auto cmp_vec2 = _mm256_cmpeq_epi8(target_vec, _mm256_loadu_si256((__m256i *)(data+i+1)));
      auto cmp_vec3 = _mm256_and_si256(cmp_vec1, cmp_vec2);
      int movemask = _mm256_movemask_epi8(cmp_vec3);
      total += _mm_popcnt_u32(movemask);
  }
  for(uint64_t i = clean_end; i < size - 1; i++) {
    if (data[i] == target && data[i + 1] == target) {
      total++;
    }
  }
  return total;
}
```

This code works by using vector operations to compare 32 bytes at a time. In the first comparision, cmp_vec1 is the result of comparing the 32 bytes from the data with a vector with 32 bytes of the target. In the second comparison, cmp_vec2 is the result of comparing the 32 bytes from the data but offseted by 1 with a vector with 32 bytes of the target. Finally, the final comparison in cmp_vec3 performs an and on the result of cmp_vec2 and cmp_vec1. This verifies that only elements where the byte is equal to the target and the next element is equal to the byte. Finally, we use movemask to convert it from a vector into a normal int and then we use popcnt to get the total bits in movemask which is equal to the number of consecutive pairs.

This code computes 32 iterations of the original loop on each iteration. The loadu instruction takes 7 cycles. The cmpeq instruction takes 1 cycle. The and instruction takes 1 cycle. The movemask instruction takes 3 cycles. The popcnt instruction takes 3 cycles. The total cost is equal to 23 cycles. 